In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

**Contents of the dataset**

* Id number: 1 to 214 (removed from CSV file)
* RI: refractive index
* Na: Sodium (unit measurement: weight percent in corresponding oxide, as are attributes 4-10)
* Mg: Magnesium
* Al: Aluminum
* Si: Silicon
* K: Potassium
* Ca: Calcium
* Ba: Barium
* Fe: Iron
* Type of glass: (class attribute) -- 
    1.  building_windows_float_processed -- 
    2. building_windows_non_float_processed -- 
    3. vehicle_windows_float_processed -- 
    4. vehicle_windows_non_float_processed (none in this database) --
    5. containers -- 
    6. tableware -- 
    7. headlamps

In [ ]:
from pandas import read_csv
data = read_csv('../input/glass.csv')
X = data.drop("Type",axis= 1)
y = data["Type"]
X.head()

In [ ]:
X.corr()

Let's import the libraries required for visualization purposes

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


**Correlation between different features**

* df.corr() compute pairwise correlation of columns.Correlation shows how the two variables are related to each other.Positive values shows as one variable increases other variable increases as well. Negative values shows as one variable increases other variable decreases.Bigger the values,more strongly two varibles are correlated and viceversa.

* We'll visualize correlation by making use of Seaborn library


In [ ]:
correlation = X.corr()
plt.figure(figsize=(10,10))
sns.heatmap(correlation, annot=True)

plt.title('Correlation between different fearures')

**Standardization of Datset**

* Standardization of a dataset makesa distribution normal with mean =0 , and SD =1. Standardization is a common method used while employing machine learning algorithms, plus they give equal independent importance to the features irrespective of what dimensions the already are in

In [ ]:
from sklearn.preprocessing import StandardScaler
X_std = StandardScaler().fit_transform(X)

**Eigen Vectors and Eigen Value Calculation**

* We create an objective function and optimize it { argmax[1/n sigma(var(uTx)^2)] } The best value of U (Ui) gives us the direction.
* While optimizing this, we generate a co-variance matrix
* (V, Lambda) : (eigen vectors, eigen values ) are obtained aftre the optimization.
* Eigen vector gives the best direction and Lambda associated to it gives an approximation of the amount of data preserved,.



In [ ]:
print('NumPy covariance matrix: \n%s' %np.cov(X_std.T)) #using its package, numpy helps us directly find the covar 
cov_matrix= np.cov(X_std.T)
print(cov_matrix.shape)
print(X.shape)


Our dataset was of the form N x k : no. of columns.  hence covar matrix should be form k x k; which is evidently true

In [ ]:
plt.figure(figsize=(8,8))
sns.heatmap(cov_matrix, annot=True)

plt.title('Correlation between different features')

**Eigen Values & Vector Calculation**


In [ ]:
eigen_vals, eigen_vects = np.linalg.eig(cov_matrix)

print('Eigenvectors \n%s' %eigen_vects)
print('\nEigenvalues \n%s' %eigen_vals)

** Selecting Principal Components**
 
 * In order to decide which eigenvectors can dropped without losing too much information for the construction of lower-dimensional subspace, we need to inspect the corresponding eigenvalues: The eigenvectors with the lowest eigenvalues bear the least information about the distribution of the data; those are the ones can be dropped.

In [ ]:
# Make a list of (eigenvalue, eigenvector) tuples
eigen_pairs = [(eigen_vals[i], eigen_vects[:,i]) for i in range(len(eigen_vals))]
eigen_pairs

# Sort the (eigenvalue, eigenvector) tuples from high to low
eigen_pairs.sort(key=lambda x: x[0], reverse=True)

Lets find the amount of data preserved in each eigen value

In [ ]:
total= sum(eigen_vals)
#this will helps us while finding the amount of data preserved in %

preserved_percent= [(i/ total)* 100  for i in sorted(eigen_vals, reverse= True)]
preserved_percent

Next, our visualization will help us analyze the amount of data preserved within all eigen_values. We'll now decide the amount of data information(variance) we decide to keep with us while performing dimensionality reduction, based on the business problem & requirements of our clients 

In [ ]:
plt.figure(figsize=(6, 4))
plt.bar(range(9), preserved_percent, alpha=0.5, align='center', label='individual explained variance')
plt.ylabel('variance percentages')
plt.xlabel('Principal components')
plt.legend(loc='best')
plt.tight_layout()

Here, The plot  shows that maximum variance (~ 26%) can be explained by the first principal component alone. The second,third,fourth and fifth principal component share almost equal amount of information.Comparatively 6th and 7th components share less amount of information as compared to the rest of the Principal components.But those information cannot be ignored since they both contribute almost 17% of the data.But we can drop the last component as it has less than 10% of the variance

For the sake of simlicity, and understanding the purpose here, let's suppose 3 principal components together comprise of 90% of variance, Hence we can drop other components. Here, we are reducing the 9-D feature space to a 3-D feature subspace, by choosing the “top 3” eigenvectors with the highest eigenvalues to construct our d×k-dimensional eigenvector matrix W

In [ ]:
matrix_v= np.hstack((eigen_pairs[0][1].reshape(9, 1), eigen_pairs[1][1].reshape(9,1), eigen_pairs[2][1].reshape(9, 1)))
#hstack stacks arrays horizontally
matrix_v

We transform our dataset into the new changed dimension: 
* F1t * v= F1' (F1:: original feature  set, F1':: new dimension)

* Here, we perform X_std (n, 9) * matrix_v (9, 3) : Y (n, 3) matrix

In [ ]:
Y= X_std.dot(matrix_v)
#dot product of the 2 vectors
print(Y)